# 0. AI 작사가 만들기

AI 작사가를 만들어보자.

## 0.1. 필요한 라이브러리 불러오기

In [1]:
import os, re
import glob

import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17924222384655484643
xla_global_id: -1
]


# 1. 데이터 읽어오기

여러 노래의 가사가 들어 있는 txt파일 49개를 전부 불러온다.

In [2]:
# colab
from google.colab import drive
drive.mount('/content/drive')
txt_file_path = '/content/drive/MyDrive/AIFFEL/Exploration/6/data/*'

# local
# txt_file_path = os.getenv('USERPROFILE') + '\OneDrive - 수원대학교\Office\AIFFEL\Exploration\\6\data\*'

print(txt_file_path)

Mounted at /content/drive
/content/drive/MyDrive/AIFFEL/Exploration/6/data/*


In [3]:
txt_list = glob.glob(txt_file_path) # 해당 경로에 있는 모든 파일들의 절대경로를 리스트에 저장.
print(txt_list[:5])

['/content/drive/MyDrive/AIFFEL/Exploration/6/data/r-kelly.txt', '/content/drive/MyDrive/AIFFEL/Exploration/6/data/radiohead.txt', '/content/drive/MyDrive/AIFFEL/Exploration/6/data/paul-simon.txt', '/content/drive/MyDrive/AIFFEL/Exploration/6/data/patti-smith.txt', '/content/drive/MyDrive/AIFFEL/Exploration/6/data/nursery_rhymes.txt']


In [4]:
raw_corpus = []

for txt_file in txt_list: # 파일경로를 하나씩
    with open(txt_file, "r", encoding='UTF8') as f: # 읽어 오는데
        sentences = f.read().splitlines() # 줄바꿈 기준으로 자른 후(1라인=1문장)
        raw_corpus.extend(sentences) # raw_corpus에 unpacking해서 저장(49개 파일의 모든 문장이 저장됨.)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['I hear you callin\', "Here I come baby"', 'To save you, oh oh', "Baby no more stallin'"]


In [5]:
# 문장 10개 둘러보기
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
    if idx > 9: break
        
    print(sentence)

I hear you callin', "Here I come baby"
To save you, oh oh
Baby no more stallin'
These hands have been longing to touch you baby
And now that you've come around, to seein' it my way
You won't regret it baby, and you surely won't forget it baby
It's unbelieveable how your body's calling for me
I can just hear it callin' callin' for me My body's callin' for you
My body's callin' for you
My body's callin' for you


# 2. 데이터 정제

In [6]:
def preprocess_sentence(sentence): # 토큰화를 하기 전 문장 정제
    sentence = sentence.lower().strip() # 전체 소문자로 변경 후 양 끝 띄어쓰기 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특정 특수문자 양쪽에 띄어쓰기 생성
    sentence = re.sub(r'[ ]+', " ", sentence) # 중복된 띄어쓰기를 하나로 변경
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 소문자, 대문자, 특정 특수문자가 아닌 것들(^ 명령어)을 띄어쓰기로 변경
    sentence = sentence.strip() # 다시 또 양 끝 띄어쓰기 제거
    sentence = '<start> ' + sentence + ' <end>'
    
    return sentence

In [7]:
print(preprocess_sentence("This @_is ;;;sample        sentence.")) # 잘 작동되는지 확인.

<start> this is sample sentence . <end>


In [8]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue # 빈 문자열 제거
        
    preprocessed_sentence = preprocess_sentence(sentence) # 정제
    if len(preprocessed_sentence.split()) <= 15:  # 토큰의 개수가 15개 이하인 문장들만.
        corpus.append(preprocessed_sentence) # 정제된 문장 저장

corpus[:10]

['<start> i hear you callin , here i come baby <end>',
 '<start> to save you , oh oh <end>',
 '<start> baby no more stallin <end>',
 '<start> these hands have been longing to touch you baby <end>',
 '<start> and now that you ve come around , to seein it my way <end>',
 '<start> it s unbelieveable how your body s calling for me <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you tell me , what s your desire <end>',
 '<start> baby your wish is my deal oh yes it is baby <end>']

## 2.1 토큰화하기

너무 긴 문장은 다른 sequence들이 과도한 Padding을 갖게 하므로 제거한다.

In [9]:
def tokenize(corpus): # 토큰화(간단히 띄어쓰기 기준으로 split한 느낌이라고 보면 됨.)
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12000, filters=' ', oov_token="<unk>") # 많이 쓰인 중복 제외 단어 12000개만 인식하고 나머지는 <unk>로 토큰화.
    tokenizer.fit_on_texts(corpus) # corpus 변수를 fitting. 단어와 인덱스를 대응시킨 딕셔너리가 생성됨.
    sequences = tokenizer.texts_to_sequences(corpus) # corpus의 모든 단어들을 인덱스로 바꾼 시퀀스 형태로 변경.
    sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences)  # 토큰이 15개 보다 적은 문장들에 패딩을 붙여줌.
    print(sequences, tokenizer)

    return sequences, tokenizer

In [10]:
sequences, tokenizer = tokenize(corpus)

[[   0    0    0 ...   68   52    3]
 [   0    0    0 ...   47   47    3]
 [   0    0    0 ...   98 6829    3]
 ...
 [   0    0    2 ...   22 4255    3]
 [   0    0    0 ... 4255  244    3]
 [   0    0    0 ...    0    2    3]] <keras_preprocessing.text.Tokenizer object at 0x7faac7a241d0>


In [11]:
print(sequences[:3, 5:])

[[   4  186    7  824    5   90    4   68   52    3]
 [   0    0    2   10  588    7    5   47   47    3]
 [   0    0    0    0    2   52   41   98 6829    3]]


In [12]:
# 가장 빈도가 높은 단어들 확인.
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [13]:
# 입력 텐서 source, 출력 텐서 target 데이터 생성.
source = sequences[:, :-1]
target = sequences[:, 1:]

print(source[0])
print(target[0])

[  0   0   0   0   2   4 186   7 824   5  90   4  68  52]
[  0   0   0   2   4 186   7 824   5  90   4  68  52   3]


# 4. 평가 데이터셋 분리

In [14]:
source_train, source_valid, target_train, target_valid = train_test_split(source, target, test_size=0.2)

In [15]:
VOCAB_SIZE = tokenizer.num_words + 1 # <pad> 까지 포함. <unk>, <start>, <end>는 이미 num_words에 포함되어 있다.

train = tf.data.Dataset.from_tensor_slices((source_train, target_train)) # 데이터 개수에 맞는 벡터들로 나눠진 source, target쌍의 tf.data.Dataset 생성. 
train = train.shuffle(len(source_train))
train = train.batch(32, drop_remainder=True) # 배치 사이즈를 1에서 32로 변경.
# valid = tf.data.Dataset.from_tensor_slices((source_valid, target_valid))

train

<BatchDataset element_spec=(TensorSpec(shape=(32, 14), dtype=tf.int32, name=None), TensorSpec(shape=(32, 14), dtype=tf.int32, name=None))>

배치 사이즈를 최초 256으로 잡고 점점 낮췄을 때 val_loss도 같이 낮아졌다.  
하지만 64에서 32로 낮췄을 때는 변화가 거의 없어서 64로 잡으려고 했는데  
GPU 할당량이 부족해서 배치사이즈를 32로 낮추었다.  
배치사이즈가 높을수록 학습이 빠르지만 그만큼 GPU를 많이 쓰고 예측 일반화 성능은 낮아질 수 있다.  
다른 시각도 존재하는데 충분히 잘 만든 모델(거의 확정적으로 global minimum에 빠지는)은  
충분한 리소스하에서는 최대한 많은 배치사이즈를 넣는 것이 배치 정규화의 효과를 더 잘 누릴 수 있다고 한다.

일반적으로 배치사이즈는 32나 64가 가장 성능이 좋다고 알려져 있다.  

또한 val_loss를 구하기 위해 valid 변수를 생성할 때 헷갈리는 점들이 몇 개 있었다.  
valid도 shuffle을 해야하는지, batch를 해야하는지 등을 찾아보았다.  
하지만 이건 검증 혹은 평가의 개념을 잘 잡았다면 헷갈릴 일이 없는 것이였다.  
애초에 valid는 학습하는 데이터가 아니기 때문에 계산만 1번 하면 되서  
shuffle도 필요 없고 batch도 필요 없는 것이었다.

하지만 결국 이것도 필요 없고 그냥 model.fit의 validation_data 파라미터에  
(source_valid, target_valid) 형식으로 집어넣는 게 가장 일반적이고 간단한 방식이라는 것을 깨달았다.

# 4. 인공지능 만들기

In [16]:
class TextGenerator(tf.keras.Model): # LSTM 모델 클래스 생성.
    def __init__(self, vocab_size, embedding_size, lstm_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) # 임베딩 레이어. 각 단어의 의미(분산 표현)를 업데이트.
        self.lstm = tf.keras.layers.LSTM(lstm_size, return_sequences=True) # LSTM 레이어. Long Short-Term Memory. RNN에 장기 의존성을 더함.
        self.linear = tf.keras.layers.Dense(vocab_size) # 출력 레이어. 열의 길이가 vocab_size가 된다.
        
    def call(self, x): # 함수처럼 쓰면 predict를 할 수 있다. 또한 fit 메소드에 이 함수가 적용된다.
        out = self.embedding(x)
        out = self.lstm(out)
        out = self.linear(out)
        
        return out

lstm 레이어를 2개 쌓았을 때보다 1개만 쌓은 모델이 더 성능이 좋게 나왔다.  
물론 GPU 할당량이 더 있었다면 레이어를 2개를 쌓고 다양한 실험을 해봤을 것이다.

In [17]:
embedding_size = 2048
lstm_size = 4096 # ★
model = TextGenerator(VOCAB_SIZE, embedding_size, lstm_size) # 모델 생성.

embedding_size를 최초 256으로 잡고 점점 늘려갔을 때 val_loss가 낮아지는 것을 볼 수 있었다.  
하지만 2048에서 4096으로 넘어갈 때는 성능차이가 거의 없었기 때문에 2048로 정했다.  
lstm_size도 마찬가지로 최초 1024로 잡고 점점 늘려갔을 때 val_loss가 낮아졌는데  
8196부터는 간혹 메모리 에러가 떠서 4096으로 잡았다.  

In [18]:
for sample in train.take(1): break

model(sample[0])

model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  24578048  
                                                                 
 lstm (LSTM)                 multiple                  100679680 
                                                                 
 dense (Dense)               multiple                  49168097  
                                                                 
Total params: 174,425,825
Trainable params: 174,425,825
Non-trainable params: 0
_________________________________________________________________


총 퍼래머더 개수가 1억 7천만개가 나왔다.

# 5. 모델 학습

In [19]:
optimizer = tf.keras.optimizers.Adam() # 최적화 도구를 adam으로 설정.
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none') # 교차 엔트로피 loss function.

model.compile(loss=loss, optimizer=optimizer)
model.fit(train, epochs=10, validation_data=(source_valid, target_valid)) # epoch마다 검증을 하기 위해 (source_valid, target_valid) 형태로 val_data 넣기.

Epoch 1/10
3905/3905 [==============================] - 821s 210ms/step - loss: 2.8159 - val_loss: 2.4808
Epoch 2/10
3905/3905 [==============================] - 829s 212ms/step - loss: 2.1136 - val_loss: 2.2057
Epoch 3/10
3905/3905 [==============================] - 830s 212ms/step - loss: 1.5992 - val_loss: 2.1337
Epoch 4/10
3905/3905 [==============================] - 829s 212ms/step - loss: 1.2897 - val_loss: 2.1737
Epoch 5/10
3905/3905 [==============================] - 829s 212ms/step - loss: 1.1508 - val_loss: 2.2321
Epoch 6/10
3905/3905 [==============================] - 828s 212ms/step - loss: 1.0956 - val_loss: 2.2902
Epoch 7/10
3905/3905 [==============================] - 827s 212ms/step - loss: 1.0655 - val_loss: 2.3515
Epoch 8/10
3905/3905 [==============================] - 828s 212ms/step - loss: 1.0501 - val_loss: 2.3879
Epoch 9/10
3905/3905 [==============================] - 828s 212ms/step - loss: 1.0417 - val_loss: 2.4384
Epoch 10/10
3905/3905 [=======================

모델을 epoch 3까지 학습한 결과 val_loss가 2.2보다 낮은 2.13을을 달성했다. 따라서 epoch 3까지만 학습한 모델을 다시 생성한다. 

In [24]:
model = TextGenerator(VOCAB_SIZE, embedding_size, lstm_size)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train, epochs=3, validation_data=(source_valid, target_valid)) # epoch마다 검증을 하기 위해 (source_valid, target_valid) 형태로 val_data 넣기.

Epoch 1/3
3905/3905 [==============================] - 876s 212ms/step - loss: 2.7881 - val_loss: 2.4614
Epoch 2/3
3905/3905 [==============================] - 830s 213ms/step - loss: 2.0652 - val_loss: 2.2202
Epoch 3/3
3905/3905 [==============================] - 831s 213ms/step - loss: 1.5655 - val_loss: 2.1559


# 6. 모델 평가

In [25]:
# 첫 단어 혹은 구절를 입력하면 문장이 끝날 때 까지 뒤에 나올 단어를 하나씩 재귀적으로 예측해주는 함수를 생성.
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence]) # 시퀀스 테스트 데이터 생성.
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64) # 텐서로 변환.
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # predict 중 가장 큰 값을 가진 인덱스를 선택.
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1) # 새로운 단어를 기존 텐서에다가 합치기.
        if predict_word.numpy()[0] == end_token: break # predict_word가 <end>면 종료.
        if test_tensor.shape[1] >= max_len: break # 최대 길이에 도달하면 강제 종료.

    generated = ""
    
    for word_index in test_tensor[0].numpy(): # 텐서를 텍스트로 변환.
        generated += tokenizer.index_word[word_index] + " "
    
    return generated

In [26]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you for your love s mine , you re my true <end> '

'I love you for your love's mine, you're my true'  
해석하면  
'내 것인 너의 사랑을 위해 너를 사랑해, 너는 나의 진실이야'  
뭔가 심오한 노랫말 같다. 외국에서는 이런 느낌의 노랫말이 많이 있는 것 같다.

In [27]:
generate_text(model, tokenizer, init_sentence="<start> i", max_len=20)

'<start> i m waiting for it , that green light , i want it <end> '

'I'm waiting for it, that green light, I want it'  
해석하면  
'나는 그것을 기다리고 있다, 그린라이트, 나는 그것을 원한다'  
그린라이트를 원하는 평범하고 일상적인 말이다.

In [28]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love me , hate me yeahh , say what you want about me <end> '

'Love me, hate me yeahh, say what you want about me'  
해석하면  
'나를 사랑해줘, 나를 미워해줘, 나에게 원하는 게 무엇인지 말해줘'  
나느 그대를 너무 사랑하는데 그대가 나를 대하는 마음은 너무나 부족해서  
나를 사랑하지 않을 거면 차라리 미워라도 해달라는 의미인 것 같다.

# 6. 회고하기

## 6.1. 이번 프로젝트에서 어려웠던 점

![](GPU%20%EC%82%AC%EC%9A%A9%EB%9F%89%20%EC%A0%9C%ED%95%9C.jpg)

GPU가 역대급으로 많이 사용 됐던 프로젝트였다. 콜랩 GPU를 너무 많이 쓴 나머지 GPU 사용량이 제한됐다.  
다양한 퍼래머더를 튜닝하기 위해 여러 세션을 동시에 돌려서 사용량이 많이 소모된 것이다.  
콜랩 pro를 썼음에도 GPU를 마음껏 쓸 수 없다는 것에 배신감을 느꼈지만 기업 입장을 생각해보니 이해는 가더라.  
차선책으로 부계정도 파봤지만 ip당 사용량을 할당하는지 부계정도 계속 제한되었다.  
Microsoft Azure Machine Learning Studio도 가입해 봤는데 얼핏보니  
GPU를 사용하는데 1시간에 8달러 정도길래 탈퇴를 알아보고 있다. 그래서 결국 LMS로 돌아오게 되었다.

## 6.2. 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점

rnn 및 lstm이 내부적으로 어떻게 돌아가는지는 잘 모르겠다. 시간이 된다면 내부 프로세스 공식 좀 알아봐야겠다.  

Tensor의 의미가 처음에는 3차원 이상의 자료구조를 부르는 건 줄 알았는데  
(스칼라(0차원 배열) - 벡터(1차원 배열) - 매트릭스(2차원 배열) - 텐서(3차원 이상의 배열))  
그냥 "자료구조" 그 자체라는 것을 알았다.  
(스칼라(0차원 텐서) - 벡터(1차원 텐서) - 매트릭스(2차원 텐서) - 3차원 이상의 텐서)  
단 2차원 이하의 텐서는 따로 부르는 명칭이 있으므로 2차원 이하에서는 텐서라고 부르는 건  
오히려 헷갈리는 것 같다. tokenize 함수를 정의할 때  
tokenizer.texts_to_sequences(corpus)의 return을 받는 변수명을 tensor로 지으셔서  
많이 헷갈렸다. 구글링의 다른 사람들은 변수명을 sequences로 지었다.

## 6.3. 루브릭 평가 지표를 맞추기 위해 시도한 것들

처음에 거의 뭐 2.8점 정도밖에 점수가 안나와서 '아... 점수 맞추기 힘들겠구나' 라는 생각을 했다.  
토큰화 했을 때 토큰의 개수가 15개 이상인 것들을 지웠고,  
Embedding size와 Hidden size를 조절했고, batch size를 조절했다.  
다만 Embedding size와 Hidden size 조절하는데 gpu 할당량을 다써서  
vocab size는 조절해보지 못한게 아쉽다.

## 6.4. 만약에 루브릭 평가 관련 지표를 달성 하지 못했을 때, 이유에 관한 추정

다행히 이번에도 평가 지표를 달성했지만 만약 gpu가 전혀 없는 환경이었다면 절대로 달성하지 못했을 것이다.

## 6.5. 회상 혹은 자기 다짐

구글 콜랩은 GPU 할당량이 정해져 있어서 파일을 마구잡이로 실행하다보면  
6시간 정도 밖에 쓰지 못하고 12시간 후에 다시 할당량이 충전된다.  
따라서 함부로 실행버튼을 누르기 보다는 지금 하고자 하는 실험을  
좀 더 명확하게 규정하고, 기록하고, 정리한 후에 실행해야겠다는 생각을 하게 되었다.

NLP가 조금 재밌어 지려고 하는 차에 GPU 부족에 막혀서 매우 아쉬웠다.